In [1]:
!which python

/u/pw7nc/anaconda3/bin/python


In [2]:
import csv
import numpy as np
from collections import Counter
from nltk.corpus import brown
from mittens import GloVe, Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import re
import json
import os
import pandas as pd
import spacy
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [3]:
def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed

def get_rareoov(xdict, val):
    return [k for (k,v) in Counter(xdict).items() if v<=val]

# Load Data

In [4]:
# Load the orignial dataset to train glove. Larger dataset can give better results for word embedding
# NOTE: For current fast development use the whole_filtered dataset.
# Later should change this to whole_cleaned dataset.

dir_path = '../Dataset/wine/'
# Load train dataset
train_review = []
cnt = 0
file_path = os.path.join(dir_path, 'whole_filtered.json')
with open(file_path) as f:
    for line in f:
        line_data = json.loads(line)
        user_id = line_data['user']
        item_id = line_data['item']
        rating = line_data['rating']
        review = line_data['review']
        train_review.append([item_id, user_id, rating, review])
        cnt += 1
        if cnt % 100000 == 0:
            print('{} lines loaded.'.format(cnt))
print('Finish loading train dataset, totally {} lines.'.format(len(train_review)))

100000 lines loaded.
200000 lines loaded.
300000 lines loaded.
Finish loading train dataset, totally 307746 lines.


In [5]:
df_train_data = pd.DataFrame(train_review, columns=['item', 'user', 'rating', 'review'])

In [6]:
train_review_doc = list(df_train_data['review'])

In [7]:
len(train_review_doc)

307746

In [8]:
with open('./embeddings/wine/train-review.txt', 'w') as f:
    cnt = 0
    for review in train_review_doc:
        # write this into file
        f.write(review.strip())
        f.write('\n')
        cnt += 1
    print('Write {} lines of review into file'.format(cnt))

Write 307746 lines of review into file


In [9]:
from collections import Counter
import simplejson
from nltk.tokenize import word_tokenize
import string
import re
import nltk
nltk.download('punkt')

def tokenize(infname="embeddings/wine/train-review.txt"):
    outfname = open(infname + ".tokenized", "w")
    sents = open(infname).read().split("\n")
    for sent in sents:
        # convert characters into lower cases
        text = sent.lower()
        # tokenize the raw text using nltk.tokenize
        text = word_tokenize(text)
        # remove all punctuation
        text = list(filter(lambda token: token not in string.punctuation, text))
        # # replace all numbers with a special token <num>
        # text = [re.sub('[0-9]+','<num>', token) for token in text]
        # write this processed text into the output file
        simplejson.dump(text, outfname)
        outfname.write("\n")
    outfname.close()


def token_filter(infname="embeddings/wine/train-review.txt.tokenized", thresh=5):
    outfname = open(infname.replace(".tokenized", ".filtered"), 'w')
    vocab = []  
    vocab_cnt = Counter()
    tokenized_sents = []
    with open(infname) as f_in:
        for line in f_in:
            tokenized_text = simplejson.loads(line)
            vocab_cnt.update(tokenized_text)
            tokenized_sents.append(tokenized_text)
    for tokenized_sent in tokenized_sents:
        # filter word that has less than 5 TF over the whole dataset
        filtered_text = []
        for token in tokenized_sent:
            if vocab_cnt[token] < thresh:
                pass
            else:
                filtered_text.append(token)
        # write this filtered data into file
        # ignore the last line of the initial imdb-small dataset since it's an empty line
        if len(tokenized_sent) > 0:
            new_sentence = " ".join(filtered_text).strip()+'\n'
            outfname.write(new_sentence)
    # bulld the vocabulary
    ## filter out token with low TF
    vocab_cnt_filter = {token: cnt for token, cnt in vocab_cnt.items() if cnt >= thresh}
    vocab_cnt_filter = Counter(vocab_cnt_filter)
    ## convert this vocabulary counter to a list
    for token,cnt in vocab_cnt_filter.most_common():
        vocab.append(token)
    outfname.close()
    return vocab

[nltk_data] Downloading package punkt to /u/pw7nc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
tokenize()
vocab = token_filter()
print("The vocab size = {}".format(len(vocab)))

The vocab size = 27729


## Load Feature Words

In [11]:
dataset_name = 'wine'
feature_2_id_file = '../Dataset/{}/train/feature/feature2id.json'.format(dataset_name)
with open(feature_2_id_file, 'r') as f:
    print("Load file: {}".format(feature_2_id_file))
    feature_vocab = json.load(f)

Load file: ../Dataset/wine/train/feature/feature2id.json


In [12]:
for fea_word in list(feature_vocab.keys()):
    if fea_word in vocab:
        pass
    else:
        print(fea_word)

gin
greengage


In [13]:
print("Length of feature vocab: {}".format(len(feature_vocab)))

Length of feature vocab: 215


In [14]:
total_vocab = set(vocab) | set(list(feature_vocab.keys()))
print("The total vocab size = {}".format(len(total_vocab)))

The total vocab size = 27731


In [15]:
total_vocab_list = list(total_vocab)

In [16]:
cv = CountVectorizer(ngram_range=(1,1), vocabulary=total_vocab_list)
X = cv.fit_transform(train_review_doc)
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

/u/pw7nc/anaconda3/lib/python3.7/site-packages/scipy/sparse/compressed.py:708: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self[i, j] = values


In [17]:
coocc_ar.shape

(27731, 27731)

In [18]:
X.shape

(307746, 27731)

In [19]:
len(cv.vocabulary_)

27731

In [20]:
train_vocab = cv.get_feature_names()

In [21]:
print(train_vocab[:20])

['warre', 'seduce', 'carnival', '.dried', 'candidates', 'pruning', 'youngish', 'reopened', 'arabic', 'soak', 'calling', 'hermitage', 'lie', 'incroyablement', 'comtes', 'conditioned', 'wore', 'union', 'declassified', 'intercontinental']


In [22]:
cv.vocabulary_["aroma"]

10639

In [27]:
train_vocab == total_vocab_list

True

In [28]:
from mittens import GloVe
import numpy as np

# mittens_model = Mittens(n=256, max_iter=1000)

# new_embeddings = mittens_model.fit(
#     coocc_ar,
#     vocab=train_vocab)

# NOTE: For fast development, set the max_iter to be 100.
# Should set it to be 1000 for more accurate word embedding.
glove_model = GloVe(n=256, max_iter=100)
embeddings = glove_model.fit(coocc_ar)

Iteration 100: error 330686.7032

In [29]:
embeddings.shape

(27731, 256)

In [30]:
len(cv.vocabulary_)

27731

In [31]:
len(cv.get_feature_names())

27731

In [32]:
vocabulary_words = cv.get_feature_names()

In [33]:
print(list(feature_vocab.keys())[:20])

['wine', 'fruit', 'palate', 'dark', 'acidity', 'cherry', 'spice', 'smooth', 'taste', 'vanilla', 'pinot', 'alcohol', 'tart', 'citrus', 'blackberry', 'plum', 'acid', 'ruby', 'lemon', 'apple']


In [34]:
print(vocabulary_words[:20])

['warre', 'seduce', 'carnival', '.dried', 'candidates', 'pruning', 'youngish', 'reopened', 'arabic', 'soak', 'calling', 'hermitage', 'lie', 'incroyablement', 'comtes', 'conditioned', 'wore', 'union', 'declassified', 'intercontinental']


In [35]:
# construct an embedding dict which has a format of {'word':'wine', 'embedding': array([...])}
word_embedding_dict = dict()
for i in range(len(embeddings)):
    current_word = vocabulary_words[i]
    current_embedding = embeddings[i]
    word_embedding_dict[current_word] = current_embedding
print("Totally {} words begin added in the word embedding dict".format(len(word_embedding_dict)))

Totally 27731 words begin added in the word embedding dict


In [36]:
# Save the Embedding dict into pickle file
import pickle
with open('./embeddings/wine/glove.pickle', 'wb') as handle:
    pickle.dump(word_embedding_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [37]:
# extract the embedding vector for each feature word
feature_embedding = dict()
emb_cnt = 0
for feature_word in list(feature_vocab.keys()):
    feature_word_id = cv.vocabulary_[feature_word]
    feature_vocab_id = feature_vocab[feature_word]
    feature_word_emb = embeddings[feature_word_id]
    feature_embedding[feature_vocab_id] = feature_word_emb
    emb_cnt += 1
print("Totally {} feature words".format(emb_cnt))

Totally 215 feature words


In [38]:
# extract the embedding vector for each feature word
feature_embed_dict = dict()
for emb_chunk in list(feature_embedding.items()):
    feature_id = emb_chunk[0]   # this is a str
    feature_emb_np = emb_chunk[1].tolist()
    feature_embed_dict[feature_id] = feature_emb_np

In [39]:
len(feature_embed_dict)

215

In [40]:
feature_emb_filepath = '../Dataset/wine/train/feature/featureid2embedding.json'
with open(feature_emb_filepath, 'w') as f:
    print("Write file: {}".format(feature_emb_filepath))
    json.dump(feature_embed_dict, f)

Write file: ../Dataset/wine/train/feature/featureid2embedding.json
